### Setup variables, paths, and create artifact registry

In [25]:
REPO_NAME="workbench"
REGION="us-central1"
PROJECT="gcp-ml-sandbox"
IMAGE_ID="python-3.9"
DERIVED_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/derived-{IMAGE_ID}"
SCRATCH_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/scratch-{IMAGE_ID}"

DERIVED_CONTAINER="derived_container"
SCRATCH_CONTAINER="scratch_container"

!mkdir {DERIVED_CONTAINER} {SCRATCH_CONTAINER}

!gcloud beta artifacts repositories create {REPO_NAME} \
    --repository-format=docker \
    --location=$REGION

mkdir: cannot create directory ‘derived_container’: File exists
mkdir: cannot create directory ‘scratch_container’: File exists
ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [66]:
!gcloud auth configure-docker us-central1-docker.pkg.dev


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [55]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       357746845324-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



### Create Docker image derived from Deeplearning containers

In [31]:
%%writefile {DERIVED_CONTAINER}/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf-gpu:latest
RUN conda create -n python-3.9
RUN conda install -n python-3.9 ipykernel python=3.9 
#SHELL ["/bin/bash", "-c"]
#RUN source /opt/conda/bin/activate /opt/conda/envs/python-3.9 && python -m ipykernel install --user --name=python-3.9 && conda deactivate


Overwriting derived_container/Dockerfile


In [32]:
!docker build . -f $DERIVED_CONTAINER/Dockerfile -t $DERIVED_IMAGE_NAME
!docker push $DERIVED_IMAGE_NAME

Sending build context to Docker daemon  14.85kB
Step 1/3 : FROM gcr.io/deeplearning-platform-release/tf-gpu:latest
 ---> fe0d6c3b8ae6
Step 2/3 : RUN conda create -n python-3.9
 ---> Using cache
 ---> f5fb1f6a4438
Step 3/3 : RUN conda install -n python-3.9 ipykernel python=3.9
 ---> Using cache
 ---> 9618a47dcbd5
Successfully built 9618a47dcbd5
Successfully tagged us-central1-docker.pkg.dev/gcp-ml-sandbox/workbench/derived-python-3.9:latest
Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/gcp-ml-sandbox/workbench/derived-python-3.9]

136e58bd: Preparing 
8e01e47a: Preparing 
e8a07179: Preparing 
36f45000: Preparing 
7e28d022: Preparing 
da7d339d: Preparing 
8cec974c: Preparing 
3ab67bd6: Preparing 
29a172ca: Preparing 
6c228da9: Preparing 
2d5f02a3: Preparing 
b8e80bc1: Preparing 
d64e4327: Preparing 
e555f78b: Preparing 
9a89d05b: Preparing 
a28a84fa: Preparing 
28ff1e27: Preparing 
02559779: Preparing 
fb637019: Preparing 
d6bc104f: Preparing 
4e7fb8

### Docker Image from scratch

In [49]:
%%writefile {SCRATCH_CONTAINER}/jupyter_notebook_config.py
c.NotebookApp.ip = '*'
c.NotebookApp.token = ''
c.NotebookApp.password = ''
c.NotebookApp.open_browser = False
c.NotebookApp.port = 8080
c.NotebookApp.terminado_settings = {'shell_command': ['/bin/bash']}
c.NotebookApp.allow_origin_pat = (
'(^https://8080-dot-[0-9]+-dot-devshell\.appspot\.com$)|'
'(^https://colab\.research\.google\.com$)|'
'((https?://)?[0-9a-z]+-dot-(?:us|asia|europe|northamerica|southamerica)-?[0-9a-z]+\.notebooks\.googleusercontent.com)')
c.NotebookApp.allow_remote_access = True
c.NotebookApp.disable_check_xsrf = False

Overwriting scratch_container/jupyter_notebook_config.py


In [64]:
%%writefile {SCRATCH_CONTAINER}/Dockerfile
FROM ubuntu:20.04

## for apt to be noninteractive
ENV DEBIAN_FRONTEND noninteractive
ENV DEBCONF_NONINTERACTIVE_SEEN true

RUN apt-get update && apt-get install -y \
    python3.9 \
    python3-pip

# Use this to set the default python to 3.9
RUN update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
RUN update-alternatives --set python3 /usr/bin/python3.9

RUN pip install jupyter -U && pip install jupyterlab

#RUN python3.9 -m pip install ipykernel && python3.9 -m ipykernel install --user --name "python-3.9"

EXPOSE 8080
             
RUN mkdir /root/.jupyter
             
COPY scratch_container/jupyter_notebook_config.py /root/.jupyter

ENV pwd=""

ENTRYPOINT exec jupyter-lab --ip=0.0.0.0 --port=8080 --no-browser --allow-root --ServerApp.allow_origin="*" --NotebookApp.token="$pwd" --NotebookApp.password="$pwd"

Overwriting scratch_container/Dockerfile


In [65]:
!docker build . -f $SCRATCH_CONTAINER/Dockerfile -t $SCRATCH_IMAGE_NAME
!docker push $SCRATCH_IMAGE_NAME

Sending build context to Docker daemon  43.01kB
Step 1/12 : FROM ubuntu:20.04
 ---> 20fffa419e3a
Step 2/12 : ENV DEBIAN_FRONTEND noninteractive
 ---> Using cache
 ---> 5ea6a8e584ee
Step 3/12 : ENV DEBCONF_NONINTERACTIVE_SEEN true
 ---> Using cache
 ---> 4aa8182fc410
Step 4/12 : RUN apt-get update && apt-get install -y     python3.9     python3-pip
 ---> Using cache
 ---> f08562219b88
Step 5/12 : RUN update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
 ---> Running in a12de5cbef3a
update-alternatives: using /usr/bin/python3.9 to provide /usr/bin/python3 (python3) in auto mode
Removing intermediate container a12de5cbef3a
 ---> 8778f2b2efa6
Step 6/12 : RUN update-alternatives --set python3 /usr/bin/python3.9
 ---> Running in 4f1ce955db21
Removing intermediate container 4f1ce955db21
 ---> b72fde2e50ac
Step 7/12 : RUN pip install jupyter -U && pip install jupyterlab
 ---> Running in eab630be8942
  Created wheel for json5: filename=json5-0.9.8-py2.py3-none-any.whl siz